In [6]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml, make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from PIL import Image
import os

from prml import nn

np.random.seed(1234)

# Function to load the images from a specified folder
def load_images_from_folder(folder_path):
    images = []
    labels = []
    for class_name in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_name)
        if os.path.isdir(class_path):
            for filename in os.listdir(class_path):
                img = Image.open(os.path.join(class_path, filename))
                img = img.resize((64, 64))
                if img is not None:
                    images.append(np.array(img))
                    labels.append(class_name)
    return np.array(images), np.array(labels)

train_dir = 'C:\\Users\\prema\\Downloads\\Machine learning\\Assignment 1\\Assignment_2\\cnn_data\\train'
test_dir = 'C:\\Users\\prema\\Downloads\\Machine learning\\Assignment 1\\Assignment_2\\cnn_data\\test'
train_images, train_labels = load_images_from_folder(train_dir)
test_images, test_labels = load_images_from_folder(test_dir)
train_images = train_images / 255.0
test_images = test_images / 255.0
label_binarizer = LabelBinarizer()
train_labels_one_hot = label_binarizer.fit_transform(train_labels)
test_labels_one_hot = label_binarizer.transform(test_labels)



In [2]:
class ConvolutionalNeuralNetwork(nn.Network):
    
    def __init__(self):
        super().__init__()
        with self.set_parameter():
            self.conv1 = nn.image.Convolve2d(
                nn.random.truncnormal(-2, 2, 1, (5, 5, 3, 20)),
                stride=(1, 1), pad=(0, 0))
            self.b1 = nn.array([0.1] * 20)
            self.conv2 = nn.image.Convolve2d(
                nn.random.truncnormal(-2, 2, 1, (5, 5, 20, 20)),
                stride=(1, 1), pad=(0, 0))
            self.b2 = nn.array([0.1] * 20)
            self.w3 = nn.random.truncnormal(-2, 2, 1, (13 * 13 * 20, 100))
            self.b3 = nn.array([0.1] * 100)
            self.w4 = nn.random.truncnormal(-2, 2, 1, (100, 3))
            self.b4 = nn.array([0.1] * 3)
        
    def __call__(self, x):
        h = nn.relu(self.conv1(x) + self.b1)
        h = nn.max_pooling2d(h, (2, 2), (2, 2))        
        h = nn.relu(self.conv2(h) + self.b2)
        h = nn.max_pooling2d(h, (2, 2), (2, 2))
        h = h.reshape(-1, 13 * 13 * 20)
        h = nn.relu(h @ self.w3 + self.b3)
        return h @ self.w4 + self.b4

model = ConvolutionalNeuralNetwork()
optimizer = nn.optimizer.Adam(model.parameter, 1e-3)

while True:
    indices = np.random.permutation(len(train_images))
    for index in range(0, len(train_images), 50):
        model.clear()
        x_batch = train_images[indices[index: index + 50]]
        y_batch = train_labels_one_hot[indices[index: index + 50]]
        logit = model(x_batch)
        log_likelihood = -nn.loss.softmax_cross_entropy(logit, y_batch).mean(0).sum()
        if optimizer.iter_count % 100 == 0:
            accuracy = accuracy_score(
                np.argmax(y_batch, axis=-1), np.argmax(logit.value, axis=-1)
            )
            print("step {:04d}".format(optimizer.iter_count), end=", ")
            print("accuracy {:.2f}".format(accuracy), end=", ")
            print("Log Likelihood {:g}".format(log_likelihood.value[0]))
        optimizer.maximize(log_likelihood)
        if optimizer.iter_count == 1000:
            break
    else:
        continue
    break

step 0000, accuracy 0.30, Log Likelihood -10957.7
step 0100, accuracy 0.34, Log Likelihood -2050.55
step 0200, accuracy 0.56, Log Likelihood -853.539
step 0300, accuracy 0.50, Log Likelihood -695.063
step 0400, accuracy 0.72, Log Likelihood -199.282
step 0500, accuracy 0.76, Log Likelihood -229.234
step 0600, accuracy 0.82, Log Likelihood -130.865
step 0700, accuracy 0.86, Log Likelihood -146.16
step 0800, accuracy 0.88, Log Likelihood -83.7727
step 0900, accuracy 0.84, Log Likelihood -73.5288


In [7]:
val_predictions = model(test_images)


val_true_labels = np.argmax(test_labels_one_hot, axis=-1)
val_pred_labels = np.argmax(val_predictions.value, axis=-1)

precision = precision_score(val_true_labels, val_pred_labels, average='macro')
recall = recall_score(val_true_labels, val_pred_labels, average='macro')
f1 = f1_score(val_true_labels, val_pred_labels, average='macro')

confusion = confusion_matrix(val_true_labels, val_pred_labels)
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1 Score: {:.2f}".format(f1))
print("Confusion Matrix:")
print(confusion)

Precision: 0.46
Recall: 0.45
F1 Score: 0.45
Confusion Matrix:
[[18  8 12]
 [ 9 15 14]
 [14  6 18]]
